In [1]:
from scrape_modules import ScrapeIshikawa
import requests, json
import slackweb
json_open = open('important.json', 'r')
important_list = json.load(json_open)

In [2]:
# データスクレイピング
sps_url = important_list['sps_url']
si = ScrapeIshikawa(sps_url)
driver = si.open_driver()
# driver = si.open_driver(False)
sikyou_url_list = si.scrape_sikyou_url(driver)
sikyou_url_list_new = si.get_sikyou_url_new(sikyou_url_list)
si.scrape_sikyou_all(driver, sikyou_url_list_new)
# si.scrape_sikyou_all(driver, sikyou_url_list)
driver.quit()

In [3]:
# 魚種列，目方列追加
df_species = si.scrape_data['銘　柄'].map(lambda x: si.container2species_size(x)[0])
df_size = si.scrape_data['銘　柄'].map(lambda x: si.container2species_size(x)[1])
si.scrape_data.insert(3, '魚種',df_species)
si.scrape_data.insert(4, '目方',df_size)

In [4]:
# データ結合
si.concat_new_data()

In [5]:
# データ確認
print(len(si.sps_data))
print(len(si.scrape_data))
print(len(si.sps_data_new))
si.sps_data_new

455
56
511


,日付,漁 法,銘 柄,魚種,目方,卸売数量[kg],高値[円/kg],平均[円/kg],安値[円/kg],1箱重量[kg]
0,2020-12-23,底曳網,あかがい(ｴｿﾞﾎﾞﾗﾓﾄﾞｷ),あかがい,-,85,1200,812,100,5
1,2020-12-23,底曳網,加能ガニ活魚(ｽﾞﾜｲｶﾞﾆ),加能ガニ活魚,-,18,25000,14125,10500,-
2,2020-12-23,底曳網,加能ガニ天2入(ｽﾞﾜｲ足折),加能ガニ天,2入,16,8500,4938,1000,4
3,2020-12-23,底曳網,加能ガニ天3入(ｽﾞﾜｲ足折),加能ガニ天,3入,16,7000,5562,2500,4
4,2020-12-23,底曳網,加能ガニ天4入(ｽﾞﾜｲ足折),加能ガニ天,4入,44,13750,7200,2500,4
...,...,...,...,...,...,...,...,...,...,...
506,2020-12-02,底曳網,加能ガニ10入(ｽﾞﾜｲｶﾞﾆ),加能ガニ,10入,80,15000,11147,6000,4
507,2020-12-02,底曳網,まだら,まだら,-,50,1900,940,400,5
508,2020-12-02,底曳網,加能ガニ8入(ｽﾞﾜｲｶﾞﾆ),加能ガニ,8入,100,25000,15861,9500,4
509,2020-12-02,底曳網,加能ガニ天8入(ｽﾞﾜｲ足折),加能ガニ天,8入,32,16250,11781,7500,4


In [6]:
# データ保存
si.save_sps('市況')

In [7]:
# 魚種に対して全サイズの平均価格を算出。
# 魚種を列，日付を行としたデータフレームを作成。
df_per_day = si.merge_per_day()
df_per_day.head(5)

,日付,あかがい,あかがれい,かわはぎ,するめいか,なめたがれい,ほっけ,まだい,まだら,みずだこ,...,はたはた,まあじ,あら,さこだい,さめ類,ぶり活〆,いぼだい,こういか,ほたるいか,なめら
0,2020-12-02,552.0,1259.0,570.5,990.000000,560.0,500.0,1258.0,940.000000,469.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-03,NaN,NaN,NaN,793.500000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-12-04,NaN,NaN,1500.0,948.333333,722.0,312.0,NaN,750.000000,1625.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-06,597.0,1165.0,NaN,681.600000,469.0,NaN,NaN,509.000000,1327.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-12-07,499.0,1005.0,1062.5,630.200000,358.0,400.0,892.0,466.666667,179.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# データ保存
si.set_with_df("by_species", df_per_day)

In [9]:
# 魚種に対して全サイズの平均価格を算出し，データフレームに格納
df_per_day_and_species = si.merge_all_per_ds()
df_per_day_and_species.head(5)
# df_per_day_and_species.fillna(0, inplace=True)

,魚種,日付,(底曳網),4入(底曳網),5入(底曳網),3入(底曳網),6入(底曳網),8入(底曳網),10入(底曳網),2入(底曳網),...,大(底曳網),雄(刺網),雄(その他),雌(その他),雌(刺網),(定置網),小(底曳網),中(底曳網),中(刺網),大(刺網)
0,あかがい,2020-12-02,552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,あかがい,2020-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,あかがい,2020-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,あかがい,2020-12-06,597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,あかがい,2020-12-07,499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# データ保存
si.set_with_df("by_species_size", df_per_day_and_species)

In [11]:
# slackに通知
content = {
    "icon_url" : "https://drive.google.com/uc?export=view&id=1bbPewWm7dHriVWqHQ2IJe3uzwbN1-RHE",
    'username': "JF石川市況",
    "text": f"JF石川の市況データを更新しました\n{important_list['sps_url']}"
    }

webhook_url = important_list["slack_webhook_url"]
requests.post(webhook_url, data = json.dumps(content))

<Response [200]>